In [1]:
import pandas as pd
import os

DATAPATH = "../data"

In [2]:
#read mollib output
mollib_10e_dict = {
    "ep2": pd.read_csv(os.path.join(DATAPATH, "mollib_10epochs", "molecules_02_0.7.txt"), header = None),
    "ep4": pd.read_csv(os.path.join(DATAPATH, "mollib_10epochs", "molecules_04_0.7.txt"), header = None),
    "ep6": pd.read_csv(os.path.join(DATAPATH, "mollib_10epochs", "molecules_06_0.7.txt"), header = None),
    "ep8": pd.read_csv(os.path.join(DATAPATH, "mollib_10epochs", "molecules_08_0.7.txt"), header = None),
    "ep10": pd.read_csv(os.path.join(DATAPATH, "mollib_10epochs", "molecules_10_0.7.txt"), header = None),
}
for k,v in mollib_10e_dict.items():
    v.columns=["smiles"]


In [3]:
for k,v in mollib_10e_dict.items():
    print(len(v))

360
330
360
322
334


In [4]:
mollib_10e = pd.concat([v for k,v in mollib_10e_dict.items()], ignore_index=True)

from rdkit import Chem

pattern_triazolopyrazine = Chem.MolFromSmarts("*-c1nnc2cncc(-*)n12")
def is_triazolopyrazine(mol):
    if mol.HasSubstructMatch(pattern_triazolopyrazine):
        return 1
    else:
        return 0

smiles = mollib_10e["smiles"].tolist()
mols = [Chem.MolFromSmiles(smi) for smi in smiles]

mollib_10e["IsTriazoloPyrazine"] = [is_triazolopyrazine(mol) for mol in (mols)]

In [5]:
idx = mollib_10e.index[(mollib_10e["IsTriazoloPyrazine"]==0)].tolist()
mollib_10e.drop(idx, inplace=True)
print(mollib_10e.shape)

(1508, 2)


In [6]:
#load already synthesized molecules and compare with mollib
gen_mols = pd.read_csv(os.path.join(DATAPATH, "data_0.csv"))
real_mols = pd.read_csv(os.path.join(DATAPATH, "training_all.csv"))

In [7]:
gen_smi = gen_mols["Smiles"].tolist()
mollib_smi = mollib_10e["smiles"].tolist()
real_smi = real_mols["smiles"].tolist()

dup_smi = list(set(gen_smi).intersection(mollib_smi))
print(len(dup_smi))
mollib_new = list(set(mollib_smi).difference(set(gen_smi)))
print(len(mollib_new))


red_smi = list(set(real_smi).intersection(set(mollib_smi)))
print(len(red_smi))
mollib_10e_new = list(set(mollib_new).difference(set(real_smi)))
print(len(mollib_10e_new))


420
823
52
771


In [9]:
mollib_data = pd.DataFrame(mollib_10e_new, columns=["smiles"])
mollib_data.to_csv(os.path.join(DATAPATH, "mollib_10e.csv"), index=False)